In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/agents-intensive-capstone-project/Hackathon dataset.txt


In [27]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [28]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types


from google.adk.agents import LlmAgent

from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [29]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [30]:
import requests

def get_top_tourist_places(location: str) -> dict:
    """
    Fetches the top 10 tourist places for a given location using Geoapify Places API.
    Always prints something, even if names are missing.
    """
    API_KEY = "8c0a4028ca814fe0b8b20ce971c4703b"
    # Step 1: Geocode the location
    print("hi")
    print("location in tourist place", location)
    geocode_url = f"https://api.geoapify.com/v1/geocode/search?text={location}&apiKey={API_KEY}"
    geocode_response = requests.get(geocode_url).json()
    
    if not geocode_response.get("features"):
        print("No coordinates found for this location.")
        return []
    
    coords = geocode_response["features"][0]["geometry"]["coordinates"]
    lon, lat = coords[0], coords[1]
    
    # Step 2: Query tourist places
    # places_url = (
    #     f"https://api.geoapify.com/v2/places?"
    #     f"categories=tourism.attraction,tourism.sights,tourism.museum&"
    #     f"filter=circle:{lon},{lat},20000&"  # expanded to 20 km radius
    #     f"limit=10&"
    #     f"apiKey={api_key}"
    # )
    places_url = (
        f"https://api.geoapify.com/v2/places?"
        # f"categories=entertainment,natural,leisure&"
        f"categories=tourism.attraction,tourism.sights&"
        # f"filter=place:51a6b8c038d867534059109f292ee9f92940f00101f9010c95780000000000c0020692030942656e67616c757275&"
        f"filter=circle:{lon},{lat},50000&"        
        f"limit=10&"
        f"apiKey={API_KEY}"
    )

    
# https://api.geoapify.com/v2/places?categories=tourism.attraction,tourism.sights&filter=circle:77.5939973522254,12.9721091,5000&bias=proximity:77.5939973522254,12.9721091&limit=20&apiKey=YOUR_API_KEY

    print(places_url)
    
    places_response = requests.get(places_url).json()
    
    # Step 3: Extract results with fallback
    tourist_places = []
    for feature in places_response.get("features", []):
        props = feature["properties"]
        name = props.get("name")
        if not name:
            # fallback: use category or generic label
            categories = props.get("categories", [])
            name = categories[0] if categories else "Tourist Place"
        address = props.get("formatted", "No address available")
        tourist_places.append({"name": name, "address": address})
    
    # Step 4: Print results
    print("Top 10 Tourist Places:")
    if not tourist_places:
        print("No tourist places found nearby.")
    else:
        for idx, place in enumerate(tourist_places, start=1):
            print(f"{idx}. {place['name']} - {place['address']}")
    
    return tourist_places


# Example usage:
# if __name__ == "__main__":
#     API_KEY = "8c0a4028ca814fe0b8b20ce971c4703b"
#     location = "Goa, India"
#     get_top_tourist_places(location, API_KEY)


Hotels agent

In [31]:
import requests
import os

RAPIDAPI_KEY = "38df62a18fmsh76c5ea622fe5734p1ccaeejsn80eba9db1c04"

# TripAdvisor API endpoints
SEARCH_LOCATION_URL = "https://tripadvisor16.p.rapidapi.com/api/v1/restaurant/searchLocation"
SEARCH_HOTELS_URL = "https://tripadvisor16.p.rapidapi.com/api/v1/hotels/searchHotels"

# def fetch_location_id(location: str):
#     """Fetch TripAdvisor location_id for a given city/location."""
#     headers = {
#         "x-rapidapi-key": RAPIDAPI_KEY,
#         "x-rapidapi-host": "tripadvisor16.p.rapidapi.com"
#     }
#     params = {"query": location}

#     response = requests.get(SEARCH_LOCATION_URL, headers=headers, params=params)
#     # print("response",response)
#     data = response.json()

#     if "data" not in data or len(data["data"]) == 0:
#         return None

#     return data["data"][0]["locationId"]

def fetch_top_hotels(location: str, checkin_date: str, checkout_date: str, max_budget_inr: int, limit: int = 10) -> dict:
    """
    Fetch top hotels for a given location & dates,
    sorted by rating (descending), filtered by budget (INR).
    """

    # location_id = fetch_location_id(location)
    location_id = "297628"
    print("hello")
    print("location_id",location_id)
    print(checkin_date,checkout_date)
    if not location_id:
        return {"error": "Unable to fetch location ID. Try another location."}

    headers = {
        "x-rapidapi-key": RAPIDAPI_KEY,
        "x-rapidapi-host": "tripadvisor16.p.rapidapi.com"
    }

    params = {
        "geoId": location_id,
        "checkIn": checkin_date,
        "checkOut": checkout_date,
        "pageNumber": "1",
        "adults": "2",
        "currency": "INR"  # Indian Rupees
    }
    print("params------",params)

    response = requests.get(SEARCH_HOTELS_URL, headers=headers, params=params)
    # print("response",response)
    data = response.json()
    print("data",data)

    if "data" not in data:
        return {"error": "No hotels found."}

    hotels = data["data"]["data"]

    filtered_hotels = []

    for h in hotels:
        price_display = h.get("priceForDisplay")  # Example: "₹12,345"
        if not price_display:
            continue

        # Clean price string -> Convert to integer
        try:
            price_clean = int(price_display.replace("₹", "").replace(",", "").strip())
        except:
            continue

        # Filter by user's max budget
        if price_clean <= max_budget_inr:
            bubble = h.get("bubbleRating", {})
            filtered_hotels.append({
                "name": h.get("title"),
                "rating": float(bubble.get("rating", 0)),
                "review_count": bubble.get("count"),
                "price": price_display,
                "numeric_price": price_clean,
                "ranking": h.get("rankingDetails", {}).get("ranking"),
                "address": h.get("geoPoint", {}),
                "photo": h.get("cardPhotos", [{}])[0].get("sizes", {}).get("medium", {}).get("url")
            })

    # Sort by rating (descending)
    filtered_sorted = sorted(filtered_hotels, key=lambda x: x["rating"], reverse=True)[:limit]

    return {
        "location": location,
        "checkin": checkin_date,
        "checkout": checkout_date,
        "budget_inr": max_budget_inr,
        "results": filtered_sorted
    }

# if __name__ == "__main__":
#     location = "bangalore"
#     checkin_date = "2025-11-30"
#     checkout_date = "2025-12-02"
#     max_budget = "20000"

#     data = fetch_top_hotels(location, checkin_date, checkout_date, max_budget_inr=max_budget, limit=10)

#     if "results" in data:
#         print(f"\nTop Hotels in {location} Under ₹{max_budget} (Sorted by Rating)\n")
#         for idx, hotel in enumerate(data["results"], 1):
#             print(f"{idx}. {hotel['name']}")
#             print(f"   Rating: {hotel['rating']}")
#             print(f"   Reviews: {hotel['review_count']}")
#             print(f"   Price: {hotel['price']}")
#             print(f"   Ranking: {hotel['ranking']}")
#             print(f"   Address Geo-Point: {hotel['address']}")
#             print(f"   Photo: {hotel['photo']}\n")
#     else:
#         print("Error:", data.get("error"))


Flight agent

In [32]:
import requests
api_key = "JYAAVKuwew0ZGVkeuwGOYASNZOZo"

def get_airline_names(carrier_codes):
    """
    Fetches full airline names for multiple carrier codes in one API call.
    """
    url = "https://test.api.amadeus.com/v1/reference-data/airlines"
    headers = {"Authorization": f"Bearer {api_key}"}
    params = {"airlineCodes": ",".join(carrier_codes)}
    
    response = requests.get(url, headers=headers, params=params)
    airline_map = {}
    
    if response.status_code == 200:
        data = response.json().get("data", [])
        for item in data:
            code = item.get("iataCode")
            name = item.get("businessName") or item.get("commonName")
            if code and name:
                airline_map[code] = name
    
    return airline_map



def find_available_flights(origin: str, destination: str, departure_date: str) -> dict:
    """
    Finds available flights with full airline names using Amadeus India live pricing.
    """
    url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
    headers = {"Authorization": f"Bearer {api_key}"}
    params = {
        "originLocationCode": origin,
        "destinationLocationCode": destination,
        "departureDate": departure_date,
        "adults": 1,
        "currencyCode": "INR",
        "max": 10
    }
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        print("Error fetching flights:", response.json())
        return []
    
    flights_data = response.json().get("data", [])
    
    if not flights_data:
        print("No flights found.")
        return []
    
    # Step 1: Collect all unique carrier codes from the response
    carrier_codes = {
        offer["itineraries"][0]["segments"][0]["carrierCode"]
        for offer in flights_data
    }
    
    # Step 2: Fetch airline names from Amadeus
    airline_map = get_airline_names(list(carrier_codes), api_key)
    
    flights = []
    for offer in flights_data:
        segment = offer["itineraries"][0]["segments"][0]
        airline_code = segment["carrierCode"]
        
        flight = {
            "airline": airline_map.get(airline_code, airline_code),
            "flight_number": segment["number"],
            "from": segment["departure"]["iataCode"],
            "to": segment["arrival"]["iataCode"],
            "departure_time": segment["departure"]["at"],
            "arrival_time": segment["arrival"]["at"],
            "duration": offer["itineraries"][0].get("duration"),
            "price": offer["price"]["total"],
            "price_currency": offer["price"]["currency"]
        }
        
        flights.append(flight)
    
    # Pretty print results
    print(f"\nAvailable flights from {origin} to {destination} on {departure_date}:\n")
    for idx, flight in enumerate(flights, start=1):
        print(
            f"{idx}. {flight['airline']} ({flight['flight_number']}) | "
            f"{flight['from']} → {flight['to']} | "
            f"{flight['departure_time']} → {flight['arrival_time']} | "
            f"₹{flight['price']}"
        )
    
    return flights



# -----------------------------------------
# SAMPLE TEST
# -----------------------------------------
# if __name__ == "__main__":
#     API_KEY = "nBB7bT7lpHBw926zsGJySCGbO6yI"
    
#     origin = "BLR"       # Bengaluru
#     destination = "DEL"  # Delhi
#     departure_date = "2025-12-01"
    
#     find_available_flights(origin, destination, departure_date, API_KEY)


In [33]:
itinerary_agent = LlmAgent(
    name="itinerary_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are a smart travel itinerary planner.

Your job is to generate a complete trip itinerary for the user:
  - Find available flights using `find_available_flights()`
  - Fetch top hotels using `fetch_top_hotels()`
  - Fetch top tourist attractions using `get_top_tourist_places()`

Follow these rules:

1. **Always call all three tools**:
    - First: `find_available_flights` using origin, destination, depart_date, return_date if provided.
    - Second: `fetch_top_hotels` using destination location, checkin_date, checkout_date.
    - Third: `get_top_tourist_places` using the same location.

2. **Check the "status" field** from each tool.
    - If any tool returns an error, explain it clearly to the user and avoid creating the itinerary.

3. **Combine outputs** from all tools into a final structured itinerary:
    - ✈️ *Flights Section:* Show best 1–3 outbound and return flight options.
    - 🏨 *Hotels Section:* Summarize top hotels with price range, rating, and location.
    - 📸 *Tourist Places Section:* Provide 5–10 top attractions with short descriptions.

4. **Format** the final answer cleanly:
    - Start with a brief summary of the trip.
    - Then present: Flights → Hotels → Tourist Places in separate sections.
    - Keep the explanations clear and concise.

5. If the user asks something unrelated to travel itineraries, politely decline
   and tell them this agent only handles itinerary planning.
""",
    tools=[
        find_available_flights,
        fetch_top_hotels,
        get_top_tourist_places
    ],
)

print("✅ Itinerary agent created with custom tools")
print("🔧 Available tools:")
print("  • find_available_flights - Returns real-time flight options")
print("  • fetch_top_hotels - Returns top hotel recommendations")
print("  • get_top_tourist_places - Returns major attractions at the location")


✅ Itinerary agent created with custom tools
🔧 Available tools:
  • find_available_flights - Returns real-time flight options
  • fetch_top_hotels - Returns top hotel recommendations
  • get_top_tourist_places - Returns major attractions at the location


In [34]:
currency_runner = InMemoryRunner(agent=itinerary_agent)
_ = await currency_runner.run_debug(
    "Create itinerary for delhi to bangalore from dates 28/11/2025 to 1/12/2025 under the budget 20000"
)


 ### Created new session: debug_session_id

User > Create itinerary for delhi to bangalore from dates 28/11/2025 to 1/12/2025 under the budget 20000


Error fetching flights: {'errors': [{'code': 38192, 'title': 'Access token expired', 'detail': 'The access token is expired', 'status': 401}]}
hello
location_id 297628
2025-11-28 2025-12-01
params------ {'geoId': '297628', 'checkIn': '2025-11-28', 'checkOut': '2025-12-01', 'pageNumber': '1', 'adults': '2', 'currency': 'INR'}
data {'status': True, 'message': 'Success', 'timestamp': 1764267005567, 'data': {'data': [{'id': '23498278', 'title': 'The Leela Bhartiya City Bengaluru', 'primaryInfo': 'Free breakfast available', 'secondaryInfo': None, 'badge': {'size': 'SMALL', 'type': 'BEST_OF_BEST', 'year': '2025'}, 'bubbleRating': {'count': '(1,537)', 'rating': 4.9}, 'isSponsored': True, 'accentedLabel': False, 'provider': 'Official Site', 'priceForDisplay': '$142', 'strikethroughPrice': None, 'priceDetails': None, 'priceSummary': 'View all 8 deals from $142', 'cardPhotos': [{'__typename': 'AppPresentation_PhotoItem', 'sizes': {'__typename': 'AppPresentation_PhotoItemSizeDynamic', 'maxHeight'

In [35]:
# import requests
# import os
# from dotenv import load_dotenv

# load_dotenv()

# RAPIDAPI_KEY = "38df62a18fmsh76c5ea622fe5734p1ccaeejsn80eba9db1c04"

# # TripAdvisor API base endpoint (RapidAPI)
# SEARCH_URL = "https://tripadvisor16.p.rapidapi.com/api/v1/restaurant/searchLocation"
# ATTRACTIONS_URL = "https://tripadvisor16.p.rapidapi.com/api/v1/attraction/searchAttractions"
# # https://tripadvisor16.p.rapidapi.com/api/v1/restaurant/searchRestaurants?locationId=304554

# def fetch_location_id(location: str):
#     """Fetch TripAdvisor location_id for a given city/location."""
#     headers = {
#         "x-rapidapi-key": RAPIDAPI_KEY,
#         "x-rapidapi-host": "tripadvisor16.p.rapidapi.com"
#     }
#     params = {"query": location}

#     response = requests.get(SEARCH_URL, headers=headers, params=params)
#     data = response.json()
#     print("data in location id",data)

#     if "data" not in data or len(data["data"]) == 0:
#         return None

#     return data["data"][0]["locationId"]
    
# def fetch_best_tourist_places(location: str, start_date: str, end_date: str, limit: int = 10):
#     """
#     Fetches best tourist attractions for a given location,
#     then sorts them by rating (descending).
#     """

#     location_id = fetch_location_id(location)
#     print("location_id",location_id)
#     if not location_id:
#         return {"error": "Unable to fetch location ID. Try another location."}

#     headers = {
#         "x-rapidapi-key": RAPIDAPI_KEY,
#         "x-rapidapi-host": "tripadvisor16.p.rapidapi.com"
#     }
#     params = {
#         "locationId": location_id,
#         "sort": "ranking",
#         "page": "1",
#         "subcategory": "all"
#     }

#     response = requests.get(ATTRACTIONS_URL, headers=headers, params=params)
#     data = response.json()

#     if "data" not in data:
#         return {"error": "No attractions found."}

#     attractions = data["data"]
#     print("attractions",attractions)

#     # Sort by rating (descending)
#     attractions_sorted = sorted(
#         attractions,
#         key=lambda x: float(x.get("rating", 0) or 0),
#         reverse=True
#     )[:limit]

#     results = []
#     for a in attractions_sorted:
#         results.append({
#             "name": a.get("name"),
#             "rating": a.get("rating"),
#             "num_reviews": a.get("numReviews"),
#             "ranking": a.get("ranking"),
#             "address": a.get("addressObj", {}).get("addressString"),
#             "photo": a.get("photo", {}).get("images", {}).get("medium", {}).get("url")
#         })

#     return {
#         "location": location,
#         "start_date": start_date,
#         "end_date": end_date,
#         "results": results
#     }

# if __name__ == "__main__":
#     location = "Bangalore"
#     start_date = "2025-11-24"
#     end_date = "2025-11-30"

#     data = fetch_best_tourist_places(location, start_date, end_date, limit=10)

#     if "results" in data:
#         print(f"\nTop 10 Tourist Places in {location} (Sorted by Rating)\n")
#         for idx, place in enumerate(data["results"], 1):
#             print(f"{idx}. {place['name']}")
#             print(f"   Rating: {place['rating']}")
#             print(f"   Reviews: {place['num_reviews']}")
#             print(f"   Ranking: {place['ranking']}")
#             print(f"   Address: {place['address']}")
#             print(f"   Photo: {place['photo']}\n")
#     else:
#         print("Error:", data.get("error"))


In [36]:
# import requests

# def find_available_flights(origin: str, destination: str, departure_date: str, api_key: str):
#     """
#     Finds available flights for the given date from origin to destination.
#     Uses Amadeus Flight Offers Search API.
    
#     Args:
#         origin (str): IATA code of origin airport (e.g., "BLR" for Bengaluru).
#         destination (str): IATA code of destination airport (e.g., "DEL" for Delhi).
#         departure_date (str): Departure date in YYYY-MM-DD format.
#         api_key (str): Your Amadeus API key (Bearer token).
    
#     Returns:
#         list: A list of flights with airline, departure, arrival, and price.
#     """
#     url = "https://test.api.amadeus.com/v2/shopping/flight-offers"

#     # api_key="j5RXpDEWPK45DZHm5TAlqmfXBHyi4Mek"
#     headers = {
#         "Authorization": f"Bearer {api_key}"
#     }
    
#     params = {
#         "originLocationCode": origin,
#         "destinationLocationCode": destination,
#         "departureDate": departure_date,
#         "adults": 1,
#         "currencyCode": "INR",
#         "max": 10  # limit to top 10 flights
#     }
    
#     response = requests.get(url, headers=headers, params=params)
    
#     if response.status_code != 200:
#         print("Error fetching flights:", response.json())
#         return []
    
#     flights_data = response.json().get("data", [])
    
#     flights = []
#     for offer in flights_data:
#         itinerary = offer["itineraries"][0]["segments"][0]
#         airline = itinerary["carrierCode"]
#         departure = itinerary["departure"]["iataCode"]
#         arrival = itinerary["arrival"]["iataCode"]
#         departure_time = itinerary["departure"]["at"]
#         arrival_time = itinerary["arrival"]["at"]
#         price = offer["price"]["total"]
        
#         flights.append({
#             "airline": airline,
#             "from": departure,
#             "to": arrival,
#             "departure_time": departure_time,
#             "arrival_time": arrival_time,
#             "price": price
#         })
    
#     print(f"Available flights from {origin} to {destination} on {departure_date}:")
#     for idx, flight in enumerate(flights, start=1):
#         print(f"{idx}. {flight['airline']} | {flight['from']} → {flight['to']} | "
#               f"{flight['departure_time']} → {flight['arrival_time']} | ₹{flight['price']}")
    
#     return flights


# # Example usage:
# if __name__ == "__main__":
#     API_KEY = "VDTAkAZRfw8Ka0MeHhxRbH5MNgGI"
#     origin = "BLR"   # Bengaluru
#     destination = "DEL"  # Delhi
#     departure_date = "2025-12-01"
    
#     find_available_flights(origin, destination, departure_date, API_KEY)


In [37]:
# import requests

# def get_airline_names(carrier_codes, api_key):
#     """
#     Fetches full airline names for multiple carrier codes in one API call.
#     """
#     url = "https://test.api.amadeus.com/v1/reference-data/airlines"
#     headers = {"Authorization": f"Bearer {api_key}"}
#     params = {"airlineCodes": ",".join(carrier_codes)}
    
#     response = requests.get(url, headers=headers, params=params)
#     airline_map = {}
    
#     if response.status_code == 200:
#         data = response.json().get("data", [])
#         for item in data:
#             code = item.get("iataCode")
#             name = item.get("businessName")
#             if code and name:
#                 airline_map[code] = name
    
#     return airline_map


# def find_available_flights(origin: str, destination: str, departure_date: str, api_key: str):
#     """
#     Finds available flights with full airline names.
#     """
#     url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
#     headers = {"Authorization": f"Bearer {api_key}"}
#     params = {
#         "originLocationCode": origin,
#         "destinationLocationCode": destination,
#         "departureDate": departure_date,
#         "adults": 1,
#         "currencyCode": "INR",
#         "max": 10
#     }
    
#     response = requests.get(url, headers=headers, params=params)
#     if response.status_code != 200:
#         print("Error fetching flights:", response.json())
#         return []
    
#     flights_data = response.json().get("data", [])
    
#     # Step 1: Collect unique carrier codes
#     carrier_codes = {offer["itineraries"][0]["segments"][0]["carrierCode"] for offer in flights_data}
    
#     # Step 2: Fetch airline names in one call
#     airline_map = get_airline_names(list(carrier_codes), api_key)
    
#     flights = []
#     for offer in flights_data:
#         itinerary = offer["itineraries"][0]["segments"][0]
#         airline_code = itinerary["carrierCode"]
#         airline_name = airline_map.get(airline_code, airline_code)  # fallback to code
#         flight_number = itinerary["number"]
#         departure = itinerary["departure"]["iataCode"]
#         arrival = itinerary["arrival"]["iataCode"]
#         departure_time = itinerary["departure"]["at"]
#         arrival_time = itinerary["arrival"]["at"]
#         price = offer["price"]["total"]
        
#         flights.append({
#             "airline": airline_name,
#             "flight_number": flight_number,
#             "from": departure,
#             "to": arrival,
#             "departure_time": departure_time,
#             "arrival_time": arrival_time,
#             "price": price
#         })
    
#     print(f"Available flights from {origin} to {destination} on {departure_date}:")
#     for idx, flight in enumerate(flights, start=1):
#         print(f"{idx}. {flight['airline']} ({flight['flight_number']}) | "
#               f"{flight['from']} → {flight['to']} | "
#               f"{flight['departure_time']} → {flight['arrival_time']} | ₹{flight['price']}")
    
#     return flights

# if __name__ == "__main__":
#     API_KEY = "VDTAkAZRfw8Ka0MeHhxRbH5MNgGI"
#     origin = "BLR"   # Bengaluru
#     destination = "DEL"  # Delhi
#     departure_date = "2025-12-01"
    
#     find_available_flights(origin, destination, departure_date, API_KEY)


In [38]:
# import requests

# def search_skyscanner_flights(origin: str, destination: str, departure_date: str, api_key: str):
#     """
#     Search flights using Skyscanner Travel API (via RapidAPI).
#     """
#     url = "https://skyscanner44.p.rapidapi.com/search"
#     headers = {
#         "X-RapidAPI-Key": api_key,
#         "X-RapidAPI-Host": "skyscanner44.p.rapidapi.com"
#     }
#     params = {
#         "origin": origin,          # e.g., BLR
#         "destination": destination, # e.g., DEL
#         "date": departure_date,     # YYYY-MM-DD
#         "currency": "INR"
#     }
    
#     response = requests.get(url, headers=headers, params=params)
#     if response.status_code != 200:
#         print("Error:", response.json())
#         return []
    
#     data = response.json()
#     flights = []
    
#     for flight in data.get("itineraries", []):
#         airline = flight.get("legs", [{}])[0].get("carriers", [{}])[0].get("name")
#         departure_time = flight.get("legs", [{}])[0].get("departure")
#         arrival_time = flight.get("legs", [{}])[0].get("arrival")
#         price = flight.get("pricing_options", [{}])[0].get("price", {}).get("amount")
        
#         flights.append({
#             "airline": airline,
#             "departure_time": departure_time,
#             "arrival_time": arrival_time,
#             "price": price
#         })
    
#     return flights


# # Example usage
# if __name__ == "__main__":
#     API_KEY = "38df62a18fmsh76c5ea622fe5734p1ccaeejsn80eba9db1c04"
#     flights = search_skyscanner_flights("BLR", "DEL", "2025-12-01", API_KEY)
#     for f in flights:
#         print(f"{f['airline']} | {f['departure_time']} → {f['arrival_time']} | ₹{f['price']}")


Flight Agent